In [1]:
import requests
from bs4 import BeautifulSoup
import pgeocode

In [2]:

print("Please enter a zipcode: ")
zipcode = input()
nomi = pgeocode.Nominatim('us')
# nomi.query_postal_code(zipcode)
longitude = str(nomi.query_postal_code(zipcode)['longitude'])
latitude = str(nomi.query_postal_code(zipcode)['latitude'])
print(longitude,latitude)


Please enter a zipcode: 
20170
-77.3675 38.9839


In [3]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat="+latitude+"&lon="+longitude)
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")


In [4]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
short_descs = [sd.get_text() for sd in seven_day.select('.tombstone-container .short-desc')]
temps = [t.get_text() for t in seven_day.select('.tombstone-container .temp')]
descs = [d['title'] for d in seven_day.select('.tombstone-container img')]



In [5]:
#Seven day forecast
import pandas as pd
weather = pd.DataFrame({
    'period':periods,
    'short_desc':short_descs,
    'temp':temps,
    'desc':descs
})
weather

,period,short_desc,temp,desc
0,Tonight,Clear,Low: 31 °F,"Tonight: Clear, with a low around 31. Northwes..."
1,Monday,Sunny,High: 47 °F,"Monday: Sunny, with a high near 47. Northwest ..."
2,MondayNight,Mostly Clear,Low: 28 °F,"Monday Night: Mostly clear, with a low around ..."
3,Tuesday,Slight ChanceRain thenRain,High: 46 °F,"Tuesday: Rain, mainly after 10am. High near 4..."
4,TuesdayNight,Rain,Low: 39 °F,"Tuesday Night: Rain, mainly before 4am. Low a..."
5,Wednesday,Partly Sunny,High: 50 °F,"Wednesday: Partly sunny, with a high near 50."
6,WednesdayNight,Mostly Clear,Low: 31 °F,"Wednesday Night: Mostly clear, with a low arou..."
7,Thursday,Sunny,High: 44 °F,"Thursday: Sunny, with a high near 44."
8,ThursdayNight,Mostly Clear,Low: 26 °F,"Thursday Night: Mostly clear, with a low aroun..."


In [6]:
#- humidity, windspeed, Dewpoint and Last Update Time
current_conditions = soup.find(id="current_conditions_detail")

soup = BeautifulSoup(page.content, 'html.parser')
humidity = soup.find_all('td')[1].get_text()
wind_speed = soup.find_all('td')[3].get_text()
dewpoint = soup.find_all('td')[7].get_text()
last_update_time = soup.find_all('td')[13].get_text().strip()

current_condition_display =pd.DataFrame([[humidity,wind_speed,dewpoint,last_update_time]], columns=['Humidity','Wind Speed','Dewpoint','Last Update Time'])
current_condition_display


,Humidity,Wind Speed,Dewpoint,Last Update Time
0,53%,NW 8 mph,23°F (-5°C),13 Nov 8:52 pm EST


In [7]:
import quandl
import pymongo
from pymongo import MongoClient

In [8]:

client = MongoClient("mongodb://localhost:27017/")
data = weather.to_dict(orient = 'records')
db = client['DS2002']
db.seven_day_forecast.insert_many(data)

In [9]:
data2 = current_condition_display.to_dict(orient = 'records')
db.current_condition.insert_many(data2)